In [2]:
!pip install pandas


  Using cached pandas-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


In [5]:
import os
from sqlalchemy import create_engine, inspect
from urllib.parse import quote_plus
import pandas as pd

# Load environment variables (optional)
# from dotenv import load_dotenv
# load_dotenv()

# PostgreSQL details
PG_HOST = os.getenv("DB_HOST", "localhost")
PG_PORT = os.getenv("DB_PORT", 5432)
PG_USER = os.getenv("DB_USER", "admin")
PG_PASSWORD = os.getenv("DB_PASSWORD", "admin@123")
PG_DB = os.getenv("DB_NAME", "sample")

# SQLite output file
SQLITE_DB = os.getenv("SQLITE_DB", "sqlite.db")

# URL-encode password in case it has special chars
pg_password_encoded = quote_plus(PG_PASSWORD)

# PostgreSQL connection
pg_url = f"postgresql+psycopg2://{PG_USER}:{pg_password_encoded}@{PG_HOST}:{PG_PORT}/{PG_DB}"
pg_engine = create_engine(pg_url)
pg_inspector = inspect(pg_engine)

# SQLite connection
sqlite_engine = create_engine(f"sqlite:///{SQLITE_DB}")

# Get all tables
tables = pg_inspector.get_table_names()

for table in tables:
    print(f"Transferring table: {table}")
    df = pd.read_sql_table(table, pg_engine)
    df.to_sql(table, sqlite_engine, index=False, if_exists="replace")

print(f"\nAll tables transferred from PostgreSQL ({PG_DB}) to SQLite ({SQLITE_DB})!")


Transferring table: allergies
Transferring table: procedures
Transferring table: providers
Transferring table: imaging_studies
Transferring table: immunizations
Transferring table: medications
Transferring table: observations
Transferring table: organizations
Transferring table: patients
Transferring table: payer_transitions
Transferring table: payers
Transferring table: careplans
Transferring table: conditions
Transferring table: devices
Transferring table: encounters

All tables transferred from PostgreSQL (sample) to SQLite (output.sqlite)!


In [8]:
import sqlite3

SQLITE_DB = "output.sqlite"

def list_tables():
    conn = sqlite3.connect(SQLITE_DB)
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in cursor.fetchall()]
        return tables
    finally:
        conn.close()

if __name__ == "__main__":
    tables = list_tables()
    print("Tables in output.sqlite:")
    for table in tables:
        print(table)


Tables in output.sqlite:
allergies
procedures
providers
imaging_studies
immunizations
medications
observations
organizations
patients
payer_transitions
payers
careplans
conditions
devices
encounters
